In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Загружаем датасеты ##

Из исправленных и очищенных файлов

### Города и поселки ###

In [2]:
# загружаем все города и поселки

all_cities = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/03_all_cities.xlsx', sheet_name='cities')

In [3]:
# загружаем ленинские города и поселки, только строго "ленинские"

lenin_cities = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/04_lenin_cities.xlsx', sheet_name='lenin_strict')

### Улицы ###

In [4]:
# загружаем все улицы

work_streets = pd.read_csv('C:/00_Projects/Lenin/Datasets/01_FIAS/05_all_streets.csv')

In [5]:
# загружаем ленинские улицы (полный вариант)

lenin_streets_full = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='full')

In [6]:
# загружаем ленинские улицы (только точные)

lenin_streets_strict = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='lenin')

In [7]:
# загружаем ленинские улицы (без переулков)

lenin_only_streets = pd.read_excel('C:/00_Projects/Lenin/Datasets/01_FIAS/06_lenin_streets.xlsx', sheet_name='lenin_os')

### Памятники ###

In [8]:
# Загружаем памятники из ОКН

monuments_OKN = pd.read_excel('C:/00_Projects/Lenin/Datasets/02_Monuments/08_lenin_reestrOKN.xlsx',
                                     sheet_name='need')

In [9]:
# Загружаем памятники из базы leninstatues

monuments_lst = pd.read_excel('C:/00_Projects/Lenin/Datasets/02_Monuments/09_lenin_monuments.xlsx',
                                     sheet_name='need')

### Названия регионов для Datawrapper ###

In [10]:
#Загружаем названия регионов для datawrapper

region_name = pd.read_excel('C:/00_Projects/Lenin/Datasets/03_Other/region_name.xlsx')

In [11]:
# создаем рабочий датафрейм

region_work = region_name[['datawrapperName', 'region']]

## Считаем и группируем ##

### Города и поселки ###

In [12]:
# считаем количество населенных пунктов в регионах

all_cities_temp = (all_cities
                   .groupby('region', as_index=False)
                   .agg({'city': 'count'})
                   .rename(columns={'city': 'total_cities'})
                  )

In [13]:
# считаем "ленинские" населенные пункты в регионах

lenin_cities_temp = (lenin_cities
                     .groupby('region', as_index=False)
                     .agg({'city': 'count'})
                     .rename(columns={'city': 'lenin_cities'})
                    )

In [14]:
# считаем типы ленинских населенных пунктов

(lenin_cities
 .groupby('city_type', as_index=False)
 .agg({'city': 'count'})
 .rename(columns={'city': 'number'})
 .sort_values('number', ascending=False)
)

#.to_excel('C:/00_Projects/Lenin/Datasets/00_for_viz/temp.xlsx', index=False)

,city_type,number
7,п,99
10,с,55
13,х,44
5,д,37
3,г,5
8,пгт,4
9,рп,4
11,с/п,3
0,арбан,1
1,аул,1


### Считаем населенные пункты с количеством улиц больше 5 и 10 ###

In [15]:
# Количество объектов (населенных пунктов) в ФИАС

all_cities.shape

(158832, 8)

In [16]:
# считаем количество улиц по населенным пунктам

city_street_number = (work_streets
                      .groupby(['region', 'mun_district', 'city', 'city_type'], as_index=False)
                      .agg({'street': 'count'})
                      .rename(columns={'street': 'street_number'})
       )

In [17]:
# количество насленных пунктов с улицами

city_street_number.shape

(90952, 5)

In [18]:
# количество объектов без улиц

all_cities.shape[0] - city_street_number.shape[0]

67880

In [19]:
# количество объектов без улиц (вычитаем из 165 тыс. - числа объектов в ФИАС до чистки)

165038 - city_street_number.shape[0]

74086

In [20]:
# считаем среднее количество улиц в населенных пунктах с улицами

city_street_number.street_number.mean()

10.725844401442519

In [21]:
# считаем медиану количества улиц в населенных пунктах с улицами

city_street_number.street_number.median()

4.0

In [22]:
city_street_number.describe()

,street_number
count,90952.000000
mean,10.725844
std,48.265112
min,1.000000
25%,1.000000
50%,4.000000
75%,8.000000
max,3521.000000


In [23]:
city_street_number.head(5)

,region,mun_district,city,city_type,street_number
0,Адыгея,Адыгейск,Адыгейск,г,63
1,Адыгея,Адыгейск,Гатлукай,аул,15
2,Адыгея,Адыгейск,Псекупс,х,20
3,Адыгея,Гиагинский,Владимировское,с,3
4,Адыгея,Гиагинский,Вольно-Веселый,х,3


In [24]:
# города и поселки с 5 и более улицами

city_more_5_streets = city_street_number.loc[city_street_number.street_number >= 5]

In [25]:
# их количество

city_more_5_streets.shape

(38711, 5)

In [26]:
# считаем количество населенных пунктов с 5 и более улицами

city_more_5_streets_temp = (city_more_5_streets
                            .groupby('region', as_index=False)
                            .agg({'city': 'count'})
                            .rename(columns={'city': 'city_m5sts'})
                            .sort_values('city_m5sts', ascending=False)
                           )
 

In [27]:
city_more_5_streets_temp.head(10)

,region,city_m5sts
6,Башкортостан,1653
42,Московская область,1471
68,Татарстан,1252
55,Ростовская область,1114
52,Пермский край,1074
31,Краснодарский край,1016
36,Ленинградская область,973
62,Свердловская область,938
2,Алтайский край,897
45,Нижегородская область,830


In [28]:
# города и поселки с 10 и более улицами

city_more_10_streets = city_street_number.loc[city_street_number.street_number >= 10]

In [29]:
# их количество

city_more_10_streets.shape

(19288, 5)

In [30]:
# считаем количество населенных пунктов с 10 и более улицами

city_more_10_streets_temp = (city_more_10_streets
                            .groupby('region', as_index=False)
                            .agg({'city': 'count'})
                            .rename(columns={'city': 'city_m10sts'})
                            .sort_values('city_m10sts', ascending=False)
                           )
 

In [31]:
city_more_10_streets_temp.head(10)

,region,city_m10sts
42,Московская область,791
31,Краснодарский край,692
6,Башкортостан,634
55,Ростовская область,629
36,Ленинградская область,597
52,Пермский край,520
62,Свердловская область,496
2,Алтайский край,482
14,Дагестан,481
68,Татарстан,477


### Считаем сумму ленинских улиц в городах с количеством улиц больше 5 и 10 ###

**Населенные пункты с количеством улиц больше 5**

In [32]:
# группируем города и поселки с ленинскими улицами

city_street_number_lsts = (lenin_streets_strict
                           .groupby(['region', 'mun_district', 'city', 'city_type'], as_index=False)
                           .agg({'street': 'count'})
                           .rename(columns={'street': 'lenin_street_number'})
                          )

In [33]:
# считаем их количество

city_street_number_lsts.shape

(8509, 5)

In [34]:
city_street_number_lsts.head()

,region,mun_district,city,city_type,lenin_street_number
0,Адыгея,Адыгейск,Адыгейск,г,1
1,Адыгея,Адыгейск,Псекупс,х,1
2,Адыгея,Гиагинский,Гиагинская,ст-ца,1
3,Адыгея,Гиагинский,Дондуковская,ст-ца,1
4,Адыгея,Кошехабльский,Блечепсин,аул,1


In [35]:
# объединяем с city_more_5_streets, чтобы отсеять нас.пункты с количеством улиц меньше 5

city_more_5_street_lsts = city_street_number_lsts.merge(
    city_more_5_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [36]:
# заполняем пропуски

city_more_5_street_lsts.fillna(0, inplace=True)

In [37]:
city_more_5_street_lsts.shape

(8509, 6)

In [38]:
# убираем нас.пункты с количеством улиц меньше 5

city_more_5_street_lsts = city_more_5_street_lsts[city_more_5_street_lsts.street_number > 0]

In [39]:
city_more_5_street_lsts.shape

(7589, 6)

In [40]:
# считаем сумму улиц в этих городах и поселках по регионам

lenin_street_sum_cm5sts_temp = (city_more_5_street_lsts
                                .groupby('region', as_index=False)
                                .agg({'lenin_street_number': 'sum'})
                                .rename(columns={'lenin_street_number': 'lenin_street_cm5st'})
                                .sort_values('lenin_street_cm5st', ascending=False)
                               )

In [41]:
lenin_street_sum_cm5sts_temp.head(10)

,region,lenin_street_cm5st
31,Краснодарский край,418
62,Свердловская область,412
68,Татарстан,355
13,Воронежская область,331
6,Башкортостан,305
55,Ростовская область,292
66,Ставропольский край,222
81,Чувашия,219
52,Пермский край,219
2,Алтайский край,213


**Населенные пункты с количеством улиц больше 10**

In [42]:
# объединяем с city_more_10_street, чтобы отсеять нас.пункты с количеством улиц меньше 5

city_more_10_street_lsts = city_street_number_lsts.merge(
    city_more_10_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [43]:
# заполняем пропуски

city_more_10_street_lsts.fillna(0, inplace=True)

In [44]:
city_more_10_street_lsts.shape

(8509, 6)

In [45]:
# убираем нас.пункты с количеством улиц меньше 10

city_more_10_street_lsts = city_more_10_street_lsts[city_more_10_street_lsts.street_number > 0]

In [46]:
city_more_10_street_lsts.shape

(5844, 6)

In [47]:
# считаем сумму улиц в этих городах и поселках по регионам

lenin_street_sum_cm10sts_temp = (city_more_10_street_lsts
                                 .groupby('region', as_index=False)
                                 .agg({'lenin_street_number': 'sum'})
                                 .rename(columns={'lenin_street_number': 'lenin_street_cm10st'})
                                 .sort_values('lenin_street_cm10st', ascending=False)
                                )

In [48]:
lenin_street_sum_cm10sts_temp.head(10)

,region,lenin_street_cm10st
31,Краснодарский край,368
62,Свердловская область,312
55,Ростовская область,256
13,Воронежская область,256
66,Ставропольский край,199
6,Башкортостан,197
52,Пермский край,180
42,Московская область,180
68,Татарстан,173
33,Крым,168


### Количество городов и поселков, в которых есть улицы Ленина ###

In [49]:
# считаем количество всех городов, в которых есть хотя бы одна улица Ленина

all_cities_lenin_streets_temp = (lenin_streets_strict
                                 .drop_duplicates(['city', 'mun_district', 'region'])
                                 .groupby('region', as_index=False)
                                 .agg({'city': 'count'})
                                 .rename(columns={'city': 'cities_1_lsts'})
                                )

Cчитаем количество городов, в которых есть несколько улиц Ленина.

In [50]:
# берем датасет lenin_only_streets, чтобы не считать переулки и группируем города по количеству улиц

all_cities_lenin_only_streets = (lenin_only_streets
                                 .groupby(['region', 'mun_district', 'city', 'city_type'], as_index=False)
                                 .agg({'street': 'count'})
                                 .rename(columns={'street': 'lenin_street_number'})
                                 .sort_values('lenin_street_number', ascending=False)
                                )

In [51]:
# смотрим среднее значение

all_cities_lenin_only_streets.lenin_street_number.mean()

1.047085201793722

In [52]:
# считаем количество городов, в которых 2 и больше улицы Ленина

all_cities_2_lenin_streets_temp = (all_cities_lenin_only_streets
                                   [all_cities_lenin_only_streets.lenin_street_number >= 2]
                                   .groupby('region', as_index=False)
                                   .agg({'city': 'count'})
                                   .rename(columns={'city': 'cities_2_lsts'})
                                   .sort_values('cities_2_lsts', ascending=False)
                                  )

### Города и поселки с улицами Ленина с общим количество улиц больше 5 и 10 ###

**Города и поселки с количеством улиц больше 5**

In [53]:
# считаем количество населенных пунктов, где есть хотя бы одна улица Ленина, с общим количеством улиц больше 5

cities_m5st_1_lst_temp = (city_more_5_street_lsts
                                .groupby('region', as_index=False)
                                .agg({'lenin_street_number': 'count'})
                                .rename(columns={'lenin_street_number': 'cities_1_lsts_cm5s'})
                                .sort_values('cities_1_lsts_cm5s', ascending=False)
                               )

In [54]:
cities_m5st_1_lst_temp.head(10)

,region,cities_1_lsts_cm5s
31,Краснодарский край,378
62,Свердловская область,374
68,Татарстан,339
6,Башкортостан,287
13,Воронежская область,279
55,Ростовская область,268
52,Пермский край,207
81,Чувашия,206
2,Алтайский край,204
33,Крым,196


In [55]:
# считаем количество населенных пунктов, где есть как минимум 2 улицы Ленина, с общим количеством улиц больше 5
# берем все города с количеством улиц больше 5
# и присоединяем к ним города с ленискими улицами в варианте без переулков - all_cities_lenin_only_streets


cities_lenin_only_streets_cm5sts = city_more_5_streets.merge(all_cities_lenin_only_streets,
                          on=(['region', 'mun_district', 'city', 'city_type']),
                              how='left'
                             )

In [56]:
# заполняем пустые значения

cities_lenin_only_streets_cm5sts.fillna(0, inplace=True)

In [57]:
cities_lenin_only_streets_cm5sts.shape

(38711, 6)

In [58]:
# удаляем нулевые строки, т.е. нас.пункты, где нет ленинских улиц

cities_lenin_only_streets_cm5sts = (cities_lenin_only_streets_cm5sts
                                    .loc[cities_lenin_only_streets_cm5sts.lenin_street_number > 0]
                                   )

In [59]:
cities_lenin_only_streets_cm5sts.shape

(7556, 6)

In [60]:
# считаем количество нас.пунктов с количеством ленинских улиц 2 и больше

cities_m5st_2_lst_temp = (cities_lenin_only_streets_cm5sts
                          .loc[cities_lenin_only_streets_cm5sts.lenin_street_number >= 2]
                          .groupby('region', as_index=False)
                          .agg({'city': 'count'})
                          .rename(columns={'city': 'cities_2_lsts_cm5s'})
                          .sort_values('cities_2_lsts_cm5s', ascending=False)
                         )

In [61]:
cities_m5st_2_lst_temp.head(10)

,region,cities_2_lsts_cm5s
36,Московская область,27
12,Воронежская область,24
39,Нижегородская область,16
49,Ростовская область,14
55,Свердловская область,14
47,Приморский край,11
58,Ставропольский край,11
7,Брянская область,10
10,Волгоградская область,9
50,Рязанская область,8


**Города и поселки с количеством улиц больше 10**

In [62]:
# считаем количество населенных пунктов, где есть хотя бы одна улица Ленина, с общим количеством улиц больше 10

cities_m10st_1_lst_temp = (city_more_10_street_lsts
                                .groupby('region', as_index=False)
                                .agg({'lenin_street_number': 'count'})
                                .rename(columns={'lenin_street_number': 'cities_1_lsts_cm10s'})
                                .sort_values('cities_1_lsts_cm10s', ascending=False)
                               )

In [63]:
cities_m10st_1_lst_temp.head(10)

,region,cities_1_lsts_cm10s
31,Краснодарский край,333
62,Свердловская область,277
55,Ростовская область,233
13,Воронежская область,204
6,Башкортостан,184
66,Ставропольский край,175
52,Пермский край,169
68,Татарстан,160
33,Крым,157
2,Алтайский край,152


In [64]:
# считаем количество населенных пунктов, где есть как минимум 2 улицы Ленина, с общим количеством улиц больше 10
# повторяем предыдущий код
# берем все города с количеством улиц больше 10
# и присоединяем к ним города с ленискими улицами в варианте без переулков - all_cities_lenin_only_streets


cities_lenin_only_streets_cm10sts = city_more_10_streets.merge(all_cities_lenin_only_streets,
                          on=(['region', 'mun_district', 'city', 'city_type']),
                              how='left'
                             )

In [65]:
# заполняем пустые значения

cities_lenin_only_streets_cm10sts.fillna(0, inplace=True)

In [66]:
cities_lenin_only_streets_cm10sts.shape

(19288, 6)

In [67]:
# удаляем нулевые строки, т.е. нас.пункты, где нет ленинских улиц

cities_lenin_only_streets_cm10sts = (cities_lenin_only_streets_cm10sts
                                    .loc[cities_lenin_only_streets_cm10sts.lenin_street_number > 0]
                                   )

In [68]:
cities_lenin_only_streets_cm10sts.shape

(5815, 6)

In [69]:
# считаем количество нас.пунктов с количеством ленинских улиц 2 и больше

cities_m10st_2_lst_temp = (cities_lenin_only_streets_cm10sts
                          .loc[cities_lenin_only_streets_cm10sts.lenin_street_number >= 2]
                          .groupby('region', as_index=False)
                          .agg({'city': 'count'})
                          .rename(columns={'city': 'cities_2_lsts_cm10s'})
                          .sort_values('cities_2_lsts_cm10s', ascending=False)
                         )

In [70]:
cities_m10st_2_lst_temp.head(10)

,region,cities_2_lsts_cm10s
36,Московская область,25
12,Воронежская область,24
39,Нижегородская область,15
49,Ростовская область,14
55,Свердловская область,14
58,Ставропольский край,11
47,Приморский край,10
7,Брянская область,10
10,Волгоградская область,9
50,Рязанская область,8


### Количество улиц ###

In [71]:
# считаем все улицы

all_streets_temp = (work_streets
                    .groupby('region', as_index=False)
                    .agg({'street': 'count'})
                    .rename(columns={'street': 'total_streets'})
                   )

In [72]:
# считаем ленинские улицы (полный вариант)

lenin_streets_full_temp = (lenin_streets_full
                           .groupby('region', as_index=False)
                           .agg({'street': 'count'})
                           .rename(columns={'street': 'len_streets_full'})
                          )

In [73]:
# считаем ленинские улицы (строгий вариант)

lenin_streets_strict_temp = (lenin_streets_strict
                             .groupby('region', as_index=False)
                             .agg({'street': 'count'})
                             .rename(columns={'street': 'len_streets_strict'})
                            )

In [74]:
# считаем главные улицы (оставляем только улицы, площади, проспекты, бульвары, шоссе)

lenin_only_streets_temp = (lenin_only_streets
                           .groupby('region', as_index=False)
                           .agg({'street': 'count'})
                           .rename(columns={'street': 'len_only_streets'})
                          )

### Памятники ###

In [75]:
# считаем памятники из ОКН

monuments_OKN_temp = (monuments_OKN
                      .groupby('region', as_index=False)
                      .agg({'monument': 'count'})
                      .rename(columns={'monument': 'monuments_OKN'})
                     )

In [76]:
# считаем памятники из базы lst

monuments_lst_temp = (monuments_lst
                      .groupby('region', as_index=False)
                      .agg({'original_name': 'count'})
                      .rename(columns={'original_name': 'monuments_lst'})
                     )

### Считаем количество памятников в населенных пунктах с количеством улиц больше 5 и 10 ###

**Больше 5 улиц в нас.пункте ОКН**

In [77]:
# группируем города и поселки с монументами из ОКН

cities_monuments_OKN = (monuments_OKN
                        .groupby(['region', 'mun_district', 'city', 'city_type'], as_index=False)
                        .agg({'monument': 'count'})
                        .rename(columns={'monument': 'monument_OKN_number'})
                       )

In [78]:
# объединяем сгруппированные нас.пункты с city_w_more_5_street, чтобы отсеять нас.пункты с количеством улиц меньше 5

city_more_5_streets_OKN = cities_monuments_OKN.merge(
    city_more_5_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [79]:
# заполняем пропуски

city_more_5_streets_OKN.fillna(0, inplace=True)

In [80]:
city_more_5_streets_OKN.shape

(944, 6)

In [81]:
# удаляем нас.пункты с количеством улиц меньше 5

city_more_5_streets_OKN = city_more_5_streets_OKN[city_more_5_streets_OKN.street_number > 0]

In [82]:
city_more_5_streets_OKN.shape

(907, 6)

In [83]:
# считаем количество памятников по регионам

monument_OKN_cm5sts_temp = (city_more_5_streets_OKN
                              .groupby('region', as_index=False)
                              .agg({'monument_OKN_number': 'count'})
                              .rename(columns={'monument_OKN_number': 'monument_OKN_cm5sts'})
                              .sort_values('monument_OKN_cm5sts', ascending=False)
                             )

In [84]:
monument_OKN_cm5sts_temp.head(10)

,region,monument_OKN_cm5sts
28,Краснодарский край,298
60,Ставропольский край,90
13,Дагестан,71
35,Мордовия,43
30,Крым,30
0,Адыгея,28
19,Калмыкия,27
49,Ростовская область,23
42,Омская область,22
3,Амурская область,20


**Больше 10 улиц в нас.пункте ОКН**

In [85]:
# объединяем сгруппированные нас.пункты с city_w_more_5_street, чтобы отсеять нас.пункты с количеством улиц меньше 5

city_more_10_streets_OKN = cities_monuments_OKN.merge(
    city_more_10_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [86]:
# заполняем пропуски

city_more_10_streets_OKN.fillna(0, inplace=True)

In [87]:
city_more_10_streets_OKN.shape

(944, 6)

In [88]:
# удаляем нас.пункты с количеством улиц меньше 10

city_more_10_streets_OKN = city_more_10_streets_OKN[city_more_10_streets_OKN.street_number > 0]

In [89]:
city_more_10_streets_OKN.shape

(832, 6)

In [90]:
# считаем количество памятников по регионам

monument_OKN_cm10sts_temp = (city_more_10_streets_OKN
                               .groupby('region', as_index=False)
                               .agg({'monument_OKN_number': 'count'})
                               .rename(columns={'monument_OKN_number': 'monument_OKN_cm10sts'})
                               .sort_values('monument_OKN_cm10sts', ascending=False)
                              )

In [91]:
monument_OKN_cm10sts_temp.head(10)

,region,monument_OKN_cm10sts
28,Краснодарский край,277
60,Ставропольский край,87
13,Дагестан,61
35,Мордовия,30
30,Крым,28
19,Калмыкия,23
49,Ростовская область,23
0,Адыгея,22
42,Омская область,21
3,Амурская область,19


**Больше 5 улиц в нас.пункте leninstatues**

In [92]:
# группируем города и поселки с монументами из leninstatues

cities_monuments_lst = (monuments_lst
                          .groupby(['region', 'mun_district', 'city', 'city_type'], as_index=False)
                          .agg({'status': 'count'})
                          .rename(columns={'status': 'monument_lst_number'})
                          .sort_values('monument_lst_number', ascending=False)
       )

In [93]:
# объединяем сгруппированные нас.пункты с city_more_5_streets, чтобы отсеять нас.пункты с количеством улиц меньше 5

city_more_5_streets_lst = cities_monuments_lst.merge(
    city_more_5_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [94]:
# заполняем пропуски

city_more_5_streets_lst.fillna(0, inplace=True)

In [95]:
city_more_5_streets_lst.shape

(3291, 6)

In [96]:
# удаляем нас.пункты с количеством улиц меньше 5

city_more_5_streets_lst = city_more_5_streets_lst[city_more_5_streets_lst.street_number > 0]

In [97]:
city_more_5_streets_lst.shape

(3090, 6)

In [98]:
# считаем количество памятников по регионам

monument_lst_cm5sts_temp = (city_more_5_streets_lst
                            .groupby('region', as_index=False)
                            .agg({'monument_lst_number': 'count'})
                            .rename(columns={'monument_lst_number': 'monument_lst_cm5sts'})
                            .sort_values('monument_lst_cm5sts', ascending=False)
                           )

In [99]:
monument_lst_cm5sts_temp.head(10)

,region,monument_lst_cm5sts
31,Краснодарский край,266
55,Ростовская область,194
42,Московская область,135
59,Саратовская область,99
48,Омская область,95
33,Крым,89
13,Воронежская область,79
45,Нижегородская область,79
66,Ставропольский край,72
35,Курская область,69


**Больше 10 улиц в нас.пункте leninstatues**

In [100]:
# объединяем сгруппированные нас.пункты с city_more_10_street, чтобы отсеять нас.пункты с количеством улиц меньше 10

city_more_10_streets_lst = cities_monuments_lst.merge(
    city_more_10_streets,
    on=['region', 'mun_district', 'city', 'city_type'], how='left')

In [101]:
# заполняем пропуски

city_more_10_streets_lst.fillna(0, inplace=True)

In [102]:
city_more_10_streets_lst.shape

(3291, 6)

In [103]:
# удаляем нас.пункты с количеством улиц меньше 10

city_more_10_streets_lst = city_more_10_streets_lst[city_more_10_streets_lst.street_number > 0]

In [104]:
city_more_10_streets_lst.shape

(2782, 6)

In [105]:
# считаем количество памятников по регионам

monument_lst_cm10sts_temp = (city_more_10_streets_lst
                            .groupby('region', as_index=False)
                            .agg({'monument_lst_number': 'count'})
                            .rename(columns={'monument_lst_number': 'monument_lst_cm10sts'})
                            .sort_values('monument_lst_cm10sts', ascending=False)
                           )

In [106]:
monument_lst_cm10sts_temp.head(10)

,region,monument_lst_cm10sts
31,Краснодарский край,248
55,Ростовская область,162
42,Московская область,120
33,Крым,78
48,Омская область,77
13,Воронежская область,75
59,Саратовская область,73
45,Нижегородская область,72
66,Ставропольский край,71
2,Алтайский край,65


## Объединяем датафреймы ##

In [107]:
# присоединяем к рабочему датафрейму количество ленинских городов и поселков (названных в честь Ильича)

region_work = region_work.merge(lenin_cities_temp, on='region', how='left')

In [108]:
# присоединяем все города и поселки

region_work = region_work.merge(all_cities_temp, on='region', how='left')

In [109]:
# присоединяем города с количеством улиц 5 и больше и 10 и больше

region_work = region_work.merge(city_more_5_streets_temp, on='region', how='left')

region_work = region_work.merge(city_more_10_streets_temp, on='region', how='left')

In [110]:
# присоединяем количество городов, где есть улицы Ленина

# хотя бы одна улица Ленина
region_work = region_work.merge(all_cities_lenin_streets_temp, on='region', how='left')

# как минимум, 2
region_work = region_work.merge(all_cities_2_lenin_streets_temp, on='region', how='left')

In [111]:
# присоединяем нас.пункты с количеством улиц больше 5 и 10, где есть улицы Ленина

# 
region_work = region_work.merge(cities_m5st_1_lst_temp, on='region', how='left')

region_work = region_work.merge(cities_m5st_2_lst_temp, on='region', how='left')


# присоединяем города с количеством улиц больше 5 и 10, где есть улицы Ленина

region_work = region_work.merge(cities_m10st_1_lst_temp, on='region', how='left')

region_work = region_work.merge(cities_m10st_2_lst_temp, on='region', how='left')

In [112]:
# присоединяем улицы

region_work = region_work.merge(all_streets_temp, on='region', how='left')


region_work = region_work.merge(lenin_streets_full_temp, on='region', how='left')


region_work = region_work.merge(lenin_streets_strict_temp, on='region', how='left')


region_work = region_work.merge(lenin_only_streets_temp, on='region', how='left')

In [113]:
# присоединяем города с количеством улиц больше 5 и 10, где есть улицы Ленина

region_work = region_work.merge(lenin_street_sum_cm5sts_temp, on='region', how='left')

region_work = region_work.merge(lenin_street_sum_cm10sts_temp, on='region', how='left')

In [114]:
# присоединяем памятники

region_work = region_work.merge(monuments_OKN_temp, on='region', how='left')

region_work = region_work.merge(monuments_lst_temp, on='region', how='left')

In [115]:
# присоединяем памятники в городах с количеством улиц больше 5 и 10

region_work = region_work.merge(monument_OKN_cm5sts_temp, on='region', how='left')

region_work = region_work.merge(monument_lst_cm5sts_temp, on='region', how='left')

region_work = region_work.merge(monument_OKN_cm10sts_temp, on='region', how='left')

region_work = region_work.merge(monument_lst_cm10sts_temp, on='region', how='left')

In [116]:
# заполняем Nan-значения

region_work.fillna(0, inplace=True)

In [117]:
region_work

,datawrapperName,region,lenin_cities,total_cities,city_m5sts,city_m10sts,cities_1_lsts,cities_2_lsts,cities_1_lsts_cm5s,cities_2_lsts_cm5s,...,len_streets_strict,len_only_streets,lenin_street_cm5st,lenin_street_cm10st,monuments_OKN,monuments_lst,monument_OKN_cm5sts,monument_lst_cm5sts,monument_OKN_cm10sts,monument_lst_cm10sts
0,Алтайский край,Алтайский край,5.0,1586,897,482,226,1.0,204,1.0,...,237,227,213,161,2.0,87,2.0,68,2.0,65
1,Амурская область,Амурская область,2.0,637,340,159,58,1.0,53,1.0,...,62,59,57,46,32.0,37,20.0,22,19.0,20
2,Архангельская область,Архангельская область,2.0,3931,353,169,33,4.0,33,4.0,...,39,39,39,37,0.0,49,0.0,31,0.0,30
3,Астраханская область,Астраханская область,1.0,437,228,170,99,6.0,93,6.0,...,112,106,106,88,1.0,38,1.0,21,1.0,19
4,Белгородская область,Белгородская область,4.0,1611,553,247,79,2.0,67,2.0,...,85,81,73,55,14.0,80,11.0,56,9.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Чеченская республика,Чечня,1.0,369,228,175,44,0.0,43,0.0,...,50,38,49,47,1.0,1,1.0,1,1.0,1
81,Чувашская республика,Чувашия,6.0,2002,626,206,277,3.0,206,3.0,...,291,281,219,125,1.0,38,1.0,21,1.0,18
82,Чукотский автономный округ,Чукотский АО,0.0,67,50,18,14,1.0,10,1.0,...,15,15,11,7,2.0,10,2.0,8,1.0,4
83,Ямало-Ненецкий автономный округ,Ямало-Ненецкий АО,0.0,130,46,32,12,0.0,12,0.0,...,14,12,14,14,0.0,4,0.0,3,0.0,3


## Записываем результат ##

In [118]:
# записываем csv

region_work.to_csv('C:/00_Projects/Lenin/Datasets/04_work_data/work_region.csv', sep=',', encoding='utf-8', index=False)